# Purpose

#### The purpose of this file is to build the database in MySQL. Both tables and views are built with SQL.

test

In [1]:
#Libraries
import mysql.connector
from mysql.connector import errorcode

## Making the Initial Connection

We need to set up the parameter names to connect to MySQL and define the name we want to call our database.

In [2]:
user = 'root'
password = 'WireTent8!'
host = 'localhost'
database = 'San_Diego_PD_Calls'

This establishes a connection to the MySQL server without specifying a database and creates a cursor object for executing SQL commands.

In [3]:
#Initial Connection
cnx = mysql.connector.connect(user=user,
                              password=password,
                              host=host,
                              )
cursor = cnx.cursor()

## Setting Up the Database

We define a function called "create_DB" to build the database if needed (in other words, if a user is running this for the first time).

In [4]:
def create_DB(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

Next, we use a try/except block to either use the existing database or execute the create_DB function to build it.

In [5]:
try:
    cursor.execute("USE {}".format(database))
    print("Using {}".format(database))
except mysql.connector.Error as err:
    print("Database {} does not exists. Will Create".format(database))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        Create_DB(cursor)
        print("Database {} created successfully.".format(database))
        cnx.database = database
    else:
        print(err)
        exit(1)

Using San_Diego_PD_Calls


# Tables

In [6]:
#initiate dictionary
TABLES = {}

#Add tables
TABLES['Days'] = (
    """
    CREATE TABLE `Days` (
        `day_of_week` enum('1','2','3','4','5','6','7') NOT NULL,
        `day` varchar(9) NOT NULL,
    PRIMARY KEY (`day_of_week`)
    )
    """
)

TABLES['Beats'] = (
    """
    CREATE TABLE `beats` (
        `beat` smallint NOT NULL,
        `neighborhood` varchar(25) NOT NULL,
    PRIMARY KEY (`beat`), 
    UNIQUE KEY `neighborhood` (`neighborhood`)
    ) 
    """
)

TABLES['Dispositions'] = (
    """
    CREATE TABLE `dispositions` (
        `dispo_code` varchar(3) NOT NULL,
        `description` tinytext NOT NULL,
    PRIMARY KEY (`dispo_code`)
    ) 
    """
)

TABLES['Call_type'] = (
    """
    CREATE TABLE `call_types` (
        `call_type` varchar(10) NULL,
        `description` tinytext NOT NULL
    ) 
    """
)

TABLES['ServiceCalls'] = (
    """
    CREATE TABLE `service_calls` (
        `incident_num` varchar(13) NOT NULL,
        `incident_date_time` datetime NOT NULL,
        `call_type` varchar(10) NULL,
        `dispo_code` varchar(3) NULL,
        `beat` smallint NOT NULL,
        `priority` tinyint NOT NULL,
        `incident_year` year NOT NULL,
    PRIMARY KEY (`incident_num`),
    CONSTRAINT `service_calls_beat` FOREIGN KEY (`beat`)
    REFERENCES `beats` (`beat`),
    CONSTRAINT `service_calls_dispo` FOREIGN KEY (`dispo_code`)
    REFERENCES `dispositions` (`dispo_code`)
    ) 
    """
)

TABLES['Address'] = (
    """
    CREATE TABLE `address` (
        `incident_num` varchar(13) NOT NULL,
        `address_number_primary` int(8)  NULL,
        `address_dir_primary` varchar(10)  NULL,
        `address_road_primary` varchar(40) NULL,
        `address_sfx_primary` varchar(10) NULL,
        `address_dir_intersecting` varchar(10) NULL,
        `address_road_intersecting` varchar(40) NULL,
        `address_sfx_intersecting` varchar(10) NULL,
        `address` varchar(80) NOT NULL,
    PRIMARY KEY (`incident_num`),
    CONSTRAINT `address_ibfk_1` 
    FOREIGN KEY (`incident_num`)
    REFERENCES `service_calls` (`incident_num`)
    ) 
    """
)

TABLES['GeoLocations'] = (
    """
    CREATE TABLE `geolocations` (
        `incident_num` varchar(13) NOT NULL,
        `lat` Decimal(8,6) NOT NULL,
        `lng` Decimal(9,6) NOT NULL,
    PRIMARY KEY (`incident_num`),
    CONSTRAINT `geolocations_ibfk_1` FOREIGN KEY (`incident_num`)
    REFERENCES `service_calls` (`incident_num`)
    ) 
    """
)

TABLES['Zipcodes'] = (
    """
    CREATE TABLE `zipcodes` (
        `incident_num` varchar(13) NOT NULL,
        `Zipcode` MEDIUMINT(5) NOT NULL,
    PRIMARY KEY (`incident_num`),
    CONSTRAINT `zipcodes_ibfk_1` 
    FOREIGN KEY (`incident_num`)
    REFERENCES `service_calls` (`incident_num`)
    ) 
    """
)



In [7]:
cursor = cnx.cursor()

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("Table has been created.")

Creating table Days: already exists.
Creating table Beats: already exists.
Creating table Dispositions: already exists.
Creating table Call_type: already exists.
Creating table ServiceCalls: already exists.
Creating table Address: already exists.
Creating table GeoLocations: already exists.
Creating table Zipcodes: already exists.


# Views

In [8]:
#initiate dictionary
VIEWS = {}

#Add VIEWS
VIEWS['Calls_Per_Day_By_Reason_Dispo_Beat'] = (
    """
    CREATE VIEW `calls_per_day_reason_disposition_beat` AS ( 
        SELECT 
            DATE(SC.INCIDENT_DATE_TIME) AS Incident_Date, 
            CT.DESCRIPTION AS Reason_For_Call, 
            D.DESCRIPTION AS Disposition, 
            B.NEIGHBORHOOD AS Beat, 
            COUNT(*) AS Total_Incidents 
        FROM 
            service_calls SC 
        INNER JOIN 
            call_types CT 
        ON 
            SC.call_type = CT.CALL_TYPE 
        INNER JOIN 
            dispositions D 
        ON 
            SC.DISPO_CODE = D.DISPO_CODE 
        INNER JOIN 
            beats B 
        ON 
            SC.BEAT = B.BEAT 
        GROUP BY 
            DATE(SC.INCIDENT_DATE_TIME), 
            CT.DESCRIPTION, 
            D.DESCRIPTION, 
            B.NEIGHBORHOOD 
    ) 
    """
)


VIEWS['CallsPerZipCode'] = (
    """
    CREATE VIEW `calls_per_zipCode` AS ( 
        SELECT 
            ZC.ZIPCODE, 
            COUNT(*) AS CALLSPERZIPCODE 
        FROM 
            service_calls SC 
        INNER JOIN 
            zipcodes ZC 
        ON 
            SC.INCIDENT_NUM = ZC.INCIDENT_NUM 
        GROUP BY 
            ZC.ZIPCODE 
    ) 
    """
)

VIEWS['CallTypesPerZipCode'] = (
    """
    CREATE VIEW `call_typess_per_zipcode` AS ( 
        SELECT 
            ZC.ZIPCODE, 
            COUNT(DISTINCT SC.CALL_TYPE) AS CALLTYPESSPERZIPCODE 
        FROM 
            service_calls SC 
        INNER JOIN 
            zipcodes ZC 
        ON 
            SC.INCIDENT_NUM = ZC.INCIDENT_NUM 
        GROUP BY 
            ZC.ZIPCODE 
    ) 
    """
)

VIEWS['CallsByBeats'] = (
    """
    CREATE VIEW `calls_beats` AS ( 
        SELECT 
            B.NEIGHBORHOOD AS Beat, 
            COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERBEAT 
        FROM 
            service_calls SC 
        INNER JOIN 
            beats B 
        ON 
            SC.BEAT = B.BEAT 
        GROUP BY 
            B.NEIGHBORHOOD 
    ) 
    """
)

VIEWS['CallsByCallType'] = (
    """
    CREATE VIEW `calls_callyype` AS ( 
        SELECT 
            CT.DESCRIPTION AS CALL_TYPE, 
            COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERCALLTYPE 
        FROM 
            service_calls SC 
        INNER JOIN 
            call_types CT 
        ON 
            SC.CALL_TYPE = CT.CALL_TYPE 
        GROUP BY 
            CT.DESCRIPTION 
    ) 
    """
)

VIEWS['CallsByDispositionCode'] = (
    """
    CREATE VIEW `calls_dispo` AS ( 
        SELECT 
            D.DESCRIPTION AS DISPOSITION, 
            COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERDISPOSITION 
        FROM 
            service_calls SC 
        INNER JOIN 
            dispositions D 
        ON 
            SC.DISPO_CODE = D.DISPO_CODE 
        GROUP BY 
            D.DESCRIPTION 
    )
    """
)

In [9]:
for view_name in VIEWS:
    view_description = VIEWS[view_name]
    try:
        print("Creating view {}: ".format(view_name), end='')
        cursor.execute(view_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("View has been created.")

Creating view Calls_Per_Day_By_Reason_Dispo_Beat: already exists.
Creating view CallsPerZipCode: already exists.
Creating view CallTypesPerZipCode: already exists.
Creating view CallsByBeats: already exists.
Creating view CallsByCallType: already exists.
Creating view CallsByDispositionCode: already exists.


In [10]:
cursor.close()
cnx.close()